In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import sklearn
from sklearn.preprocessing import StandardScaler

In [2]:
dataset = pd.read_excel("../data/acp_data.xls")

In [3]:
dataset.head(5)

,Modele,CYL,PUISS,LONG,LARG,POIDS,V-MAX,FINITION,PRIX,R-POID.PUIS
0,Alfasud TI,1350,79,393,161,870,165,B,30570,11.012658
1,Audi 100,1588,85,468,177,1110,160,TB,39990,13.058824
2,Simca 1300,1294,68,424,168,1050,152,M,29600,15.441176
3,Citroen GS Club,1222,59,412,161,930,151,M,28250,15.762712
4,Fiat 132,1585,98,439,164,1105,165,B,34900,11.275510


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 10 columns):
Modele         18 non-null object
CYL            18 non-null int64
PUISS          18 non-null int64
LONG           18 non-null int64
LARG           18 non-null int64
POIDS          18 non-null int64
V-MAX          18 non-null int64
FINITION       18 non-null object
PRIX           18 non-null int64
R-POID.PUIS    18 non-null float64
dtypes: float64(1), int64(7), object(2)
memory usage: 1.5+ KB


In [5]:
dataset.dtypes

Modele          object
CYL              int64
PUISS            int64
LONG             int64
LARG             int64
POIDS            int64
V-MAX            int64
FINITION        object
PRIX             int64
R-POID.PUIS    float64
dtype: object

In [6]:
dataset.shape

(18, 10)

In [7]:
#nombre d'observations
dataset.shape[0]

18

In [8]:
#nombre de variables
dataset.shape[1]

10

In [9]:
#instanciation
sc = StandardScaler()

In [10]:
categorical_features = dataset[['FINITION', 'Modele']]
categorical_features 

,FINITION,Modele
0,B,Alfasud TI
1,TB,Audi 100
2,M,Simca 1300
3,M,Citroen GS Club
4,B,Fiat 132
5,TB,Lancia Beta
6,B,Peugeot 504
7,B,Renault 16 TL
8,TB,Renault 30
9,M,Toyota Corolla


In [11]:
dataset = pd.concat([dataset, pd.get_dummies(dataset['FINITION'])], axis=1);

In [12]:
dataset = pd.concat([dataset, pd.get_dummies(dataset['Modele'])], axis=1);

In [13]:
dataset.dtypes

Modele              object
CYL                  int64
PUISS                int64
LONG                 int64
LARG                 int64
POIDS                int64
V-MAX                int64
FINITION            object
PRIX                 int64
R-POID.PUIS        float64
B                    uint8
M                    uint8
TB                   uint8
Alfasud TI           uint8
Alfetta-1.66         uint8
Audi 100             uint8
Citroen GS Club      uint8
Datsun-200L          uint8
Fiat 132             uint8
Lada-1300            uint8
Lancia Beta          uint8
Mazda-9295           uint8
Opel-Rekord          uint8
Peugeot 504          uint8
Princess-1800        uint8
Rancho               uint8
Renault 16 TL        uint8
Renault 30           uint8
Simca 1300           uint8
Taunus-2000          uint8
Toyota Corolla       uint8
dtype: object

In [14]:
dataset= dataset.drop(['FINITION', 'PRIX', 'R-POID.PUIS', 'Modele'], axis=1)

In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 27 columns):
CYL                18 non-null int64
PUISS              18 non-null int64
LONG               18 non-null int64
LARG               18 non-null int64
POIDS              18 non-null int64
V-MAX              18 non-null int64
B                  18 non-null uint8
M                  18 non-null uint8
TB                 18 non-null uint8
Alfasud TI         18 non-null uint8
Alfetta-1.66       18 non-null uint8
Audi 100           18 non-null uint8
Citroen GS Club    18 non-null uint8
Datsun-200L        18 non-null uint8
Fiat 132           18 non-null uint8
Lada-1300          18 non-null uint8
Lancia Beta        18 non-null uint8
Mazda-9295         18 non-null uint8
Opel-Rekord        18 non-null uint8
Peugeot 504        18 non-null uint8
Princess-1800      18 non-null uint8
Rancho             18 non-null uint8
Renault 16 TL      18 non-null uint8
Renault 30         18 non-null uint8
Simca 130

In [16]:
#transformation–centrage-réduction
Z = sc.fit_transform(dataset)
print(Z)

[[-0.77509889 -0.28335818 -1.88508077 -1.09734528 -1.56900676  0.56976043
   1.25356634 -0.62017367 -0.70710678  4.12310563 -0.24253563 -0.24253563
  -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563
  -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563
  -0.24253563 -0.24253563 -0.24253563]
 [-0.12016326  0.01963869  1.60580955  2.0010414   0.23416142  0.14597168
  -0.79772404 -0.62017367  1.41421356 -0.24253563 -0.24253563  4.12310563
  -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563
  -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563
  -0.24253563 -0.24253563 -0.24253563]
 [-0.92920139 -0.83885242 -0.44217944  0.25819889 -0.21663062 -0.53209032
  -0.79772404  1.61245155 -0.70710678 -0.24253563 -0.24253563 -0.24253563
  -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563
  -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563 -0.24253563
   4.12310563 -0.24253563 -0.24253

In [18]:
#Moyenne
print(np.mean(Z,axis=0))

[-2.22044605e-16 -1.41861831e-16  0.00000000e+00  1.86270752e-15
  5.73615229e-16  5.55111512e-16 -6.16790569e-18  0.00000000e+00
  7.40148683e-17 -1.23358114e-17 -1.23358114e-17 -1.23358114e-17
 -1.23358114e-17 -1.23358114e-17 -1.23358114e-17  0.00000000e+00
 -1.23358114e-17 -1.23358114e-17  6.16790569e-18 -1.23358114e-17
 -1.23358114e-17 -1.23358114e-17 -1.23358114e-17 -1.23358114e-17
 -1.23358114e-17 -1.23358114e-17 -1.23358114e-17]


In [20]:
#Écart types
print(np.std(Z,axis=0,ddof=0))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]
